# MVP - Engenharia de Dados
## Autor: Marcel Antonio Lopes Pereira

## Objetivo:

Esse MVP consiste na construção de um pipeline de dados utilizando tecnologias na nuvem. O pipeline envolve a busca, coleta, modelagem, carga e análise dos dados para responder a perguntas relativas a solução de um problema.

A questão principal motivadora para esse trabalho é: Existe alguma seleção nacional masculina dominante na história do futebol a ponto de ser considerada a melhor da História até os dias de hoje? 

Essa é uma pergunta que motiva muitas discussões entre torcedores por todo o mundo. O desempenho em Copas do Mundo pode ser um bom indicativo e isso pode ser verificado analisando os dados. Além disso, são levantadas nesse processo algumas outras questões cujas respostas podem auxiliar na busca pela resposta da pergunta inicial:
- Qual seleção tem o maior número de vitórias em jogos oficiais e amistosos?
- Qual seleção tem a melhor taxa de aproveitamento (pontos ganhos por jogo) em toda a história?
- Qual seleção tem o maior saldo de gols (gols marcados - gols sofridos) em toda a história?
- Qual seleção tem o maior número de títulos oficiais (Copas do Mundo, Copas Continentais, etc.)?
- Qual seleção teve a maior sequência invicta em toda a história?
- Qual seleção tem o maior número de gols marcados em toda a história?
- Qual seleção tem a menor média de gols sofridos por jogo em toda a história?
- Qual seleção tem o maior número de vitórias em jogos decisivos (finais, semifinais, etc.)? 
- Qual seleção tem o maior número de jogos sem sofrer gols em toda a história?
- Qual seleção tem o maior número de vitórias contra outras grandes seleções (top 10 do ranking da FIFA)?
- Qual seleção tem o melhor desempenho como mandante e como visitante?
- Qual seleção tem o maior número de vitórias por diferença de gols (goleadas)?
- Qual seleção tem o maior número de vitórias em jogos decididos nos acréscimos ou na prorrogação?
- Qual seleção tem o maior número de vitórias em jogos contra a atual campeã mundial?
- Qual seleção tem o maior número de vitórias em jogos como visitante contra seleções do top 10 do ranking da FIFA?

O objetivo desse trabalho é realizar uma busca nos dados para tentar encontrar as respostas para todas essas perguntas. 

Este notebook é o responsável pela coleta dos dados e construção da camada bronze, com os dados brutos, conforme estruturados nos arquivos originais. As camadas silver e gold são construídas em outros notebooks associados a esse trabalho. Por fim, há um outro notebook onde a análise dos dados e as perguntas do problemas serão respondidas, bem como a conclusão do trabalho.




## Busca dos dados:

Na busca dos dados necessários para solução do problema, foi escolhida o Kaggle como fonte para obtenção dos mesmos. Nesse processo, foram escolhidos dois conjuntos de dados que são descritos a seguir.

O primeiro conjunto é referente aos resultados de partidas entre seleções no período entre 1872 e 2024. Nesses dados não estão incluídos os resultados de partidas em Jogos Olímípicos, com seleções nacionais B, sub-23 e seleções com melhores jogadores de ligas. O link de acesso a esse conjunto de dados é o seguinte:
https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017

Esses dados foram publicados pelo usuário Mart Jürisoo e são de domínio público (CC0: Public Domain). O conjunto possui 4 arquivos do tipo csv: results.csv (resultados das partidas); shootouts.csv (disputas de penalties); goalscorers.csv (marcadores de gols); former_names.csv (nomes anteriores das equipes).

O segundo conjunto é referente ao Ranking da FIFA de seleções no período de 1992 a 2024. Estão listadas as posições dos times e suas mudanças ao longo do tempo dentro desse intervalo. O link de acesso a esse conjunto de dados é o seguinte:
https://www.kaggle.com/datasets/cashncarry/fifaworldranking

Esses dados forma publicados pelo usuário Alex e também são de domínio público (CC0: Public Domain). Apesar do conjunto possuir 3 arquivos do tipo csv, somente foi utilizado um deles nesse trabalho: fifa_ranking-2024-06-20.csv (Lista de Rankings até 20-06-24).

## Coleta, modelagem e carga dos dados:

Após identificar os dados necessários para o problema, é realizada a coleta e a carga dos mesmos para o ambiente de nuvem. Foi utilizado nesse processo o sistema de armazenamento do Datrabricks com utilização do framework Spark para carga dos dados.


### Camada Bronze

Na camada bronze, são importados os dados no seu formato bruto a partir dos arquivos csv de origem. São realizados scripts relativos a criação da database onde serão armazenados as tabelas da camada bronze.

In [0]:
%sql DROP DATABASE bronze;

In [0]:
%sql CREATE DATABASE bronze;

Após a criação do database, os arquivos csv são carregados no sistema de armazenamento do Databricks através de uma opção de upload disponível no sistema. Após esse upload, o framework Spark realiza a leitura desse arquivo para uma visualização preliminar dos dados carregados. 

In [0]:
# Carga da tabela de resultados das partidas
file_location = "/FileStore/tables/results.csv"
file_type = "csv"

# Opções CSV
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Importando o CSV para o dataframe df_results.
df_results = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Visualizando os dados carregados
display(df_results.show(10))

+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  false|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  false|
|1874-03-07| Scotland|  England|         2|         1|  Friendly|Glasgow|Scotland|  false|
|1875-03-06|  England| Scotland|         2|         2|  Friendly| London| England|  false|
|1876-03-04| Scotland|  England|         3|         0|  Friendly|Glasgow|Scotland|  false|
|1876-03-25| Scotland|    Wales|         4|         0|  Friendly|Glasgow|Scotland|  false|
|1877-03-03|  England| Scotland|         1|         3|  Friendly| London| England|  false|
|1877-03-05|    Wales| Scotland|         0|         2|  Friendly|Wrexham|   Wales|  false|

In [0]:
# Salvar o DataFrame como uma tabela na camada bronze
df_results.write.mode("overwrite").saveAsTable("bronze.results")

A seguir, o mesmo procedimento é realizado para carregar os respectivos arquivos csv das outras tabelas e salvamento das mesmas como tabelas na camada Bronze.

In [0]:
# Caminho do arquivo CSV
file_location = "/FileStore/tables/shootouts.csv"
file_type = "csv"

# Opções CSV
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Carregando o CSV para o DataFrame df_shootouts
df_shootouts = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Visualizando os dados carregados
display(df_shootouts.show(10))

# Salvando na camada bronze
df_shootouts.write.mode("overwrite").saveAsTable("bronze.shootouts")

+----------+-----------+----------------+-----------+-------------+
|      date|  home_team|       away_team|     winner|first_shooter|
+----------+-----------+----------------+-----------+-------------+
|1967-08-22|      India|          Taiwan|     Taiwan|         null|
|1971-11-14|South Korea|Vietnam Republic|South Korea|         null|
|1972-05-07|South Korea|            Iraq|       Iraq|         null|
|1972-05-17|   Thailand|     South Korea|South Korea|         null|
|1972-05-19|   Thailand|        Cambodia|   Thailand|         null|
|1973-04-21|    Senegal|           Ghana|      Ghana|         null|
|1973-06-14|     Guinea|            Mali|     Guinea|         null|
|1973-06-14|  Mauritius|        Tanzania|  Mauritius|         null|
|1973-07-26|   Malaysia|          Kuwait|   Malaysia|         null|
|1973-07-26|   Cambodia|       Singapore|  Singapore|         null|
+----------+-----------+----------------+-----------+-------------+
only showing top 10 rows



In [0]:
# Caminho do arquivo CSV
file_location = "/FileStore/tables/goalscorers.csv"
file_type = "csv"

# Opções CSV
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Carregando o CSV para o DataFrame df_goalscorers
df_goalscorers = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Visualizando os dados carregados
display(df_goalscorers.show(10))

# Salvando na camada bronze
df_goalscorers.write.mode("overwrite").saveAsTable("bronze.goalscorers")

+----------+---------+---------+---------+--------------------+------+--------+-------+
|      date|home_team|away_team|     team|              scorer|minute|own_goal|penalty|
+----------+---------+---------+---------+--------------------+------+--------+-------+
|1916-07-02|    Chile|  Uruguay|  Uruguay|     José Piendibene|    44|   false|  false|
|1916-07-02|    Chile|  Uruguay|  Uruguay|    Isabelino Gradín|    55|   false|  false|
|1916-07-02|    Chile|  Uruguay|  Uruguay|    Isabelino Gradín|    70|   false|  false|
|1916-07-02|    Chile|  Uruguay|  Uruguay|     José Piendibene|    75|   false|  false|
|1916-07-06|Argentina|    Chile|Argentina|       Alberto Ohaco|     2|   false|  false|
|1916-07-06|Argentina|    Chile|    Chile|      Telésforo Báez|    44|   false|  false|
|1916-07-06|Argentina|    Chile|Argentina|  Juan Domingo Brown|    60|   false|   true|
|1916-07-06|Argentina|    Chile|Argentina|  Juan Domingo Brown|    62|   false|   true|
|1916-07-06|Argentina|    Chile|

In [0]:
# Caminho do arquivo CSV
file_location = "/FileStore/tables/former_names.csv"
file_type = "csv"

# Opções CSV
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Carregar o CSV para o DataFrame df_former_names
df_former_names = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Visualizando os dados carregados
display(df_former_names.show(10))

# Salvando na camada bronze
df_former_names.write.mode("overwrite").saveAsTable("bronze.former_names")

+--------------+--------------------+----------+----------+
|       current|              former|start_date|  end_date|
+--------------+--------------------+----------+----------+
|         Benin|             Dahomey|1959-11-08|1975-11-30|
|  Burkina Faso|         Upper Volta|1960-04-14|1984-08-04|
|       Curaçao|Netherlands Antilles|1957-03-03|2010-10-10|
|Czechoslovakia|             Bohemia|1903-04-05|1919-01-01|
|Czechoslovakia| Bohemia and Moravia|1939-01-01|1940-01-01|
|      DR Congo|       Belgian Congo|1948-05-25|1956-01-02|
|      DR Congo|  Congo-Léopoldville|1963-04-12|1964-07-19|
|      DR Congo|      Congo-Kinshasa|1965-01-09|1970-11-24|
|      DR Congo|               Zaïre|1971-01-10|1997-04-27|
|      Djibouti|   French Somaliland|1947-12-05|1977-06-27|
+--------------+--------------------+----------+----------+
only showing top 10 rows



In [0]:
# Caminho do arquivo CSV
file_location = "/FileStore/tables/fifa_ranking_2024_06_20.csv"
file_type = "csv"

# Opções CSV
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# Carregar o CSV para o DataFrame df_fifa_ranking
df_fifa_ranking = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

# Visualizando os dados carregados
display(df_fifa_ranking.show(10))

# Salvando na camada bronze
df_fifa_ranking.write.mode("overwrite").saveAsTable("bronze.fifa_ranking")

+-----+-----------------+------------+------------+---------------+-----------+-------------+----------+
| rank|     country_full|country_abrv|total_points|previous_points|rank_change|confederation| rank_date|
+-----+-----------------+------------+------------+---------------+-----------+-------------+----------+
|140.0|Brunei Darussalam|         BRU|         2.0|            0.0|        140|          AFC|1992-12-31|
| 33.0|         Portugal|         POR|        38.0|            0.0|         33|         UEFA|1992-12-31|
| 32.0|           Zambia|         ZAM|        38.0|            0.0|         32|          CAF|1992-12-31|
| 31.0|           Greece|         GRE|        38.0|            0.0|         31|         UEFA|1992-12-31|
| 30.0|          Algeria|         ALG|        39.0|            0.0|         30|          CAF|1992-12-31|
| 29.0|       Yugoslavia|         YUG|        39.0|            0.0|         29|         UEFA|1992-12-31|
| 28.0|            Wales|         WAL|        40.0|    

Após a execução dos scripts acima, a camada bronze foi construída com as tabelas. Em seguida, são realizadas consultas SQL para verificar os dados. 


In [0]:
%sql SELECT * FROM bronze.results LIMIT 10

date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,false
1873-03-08,England,Scotland,4,2,Friendly,London,England,false
1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,false
1875-03-06,England,Scotland,2,2,Friendly,London,England,false
1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,false
1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,false
1877-03-03,England,Scotland,1,3,Friendly,London,England,false
1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,false
1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,false
1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,false


In [0]:
%sql SELECT * FROM bronze.shootouts LIMIT 10

date,home_team,away_team,winner,first_shooter
1967-08-22,India,Taiwan,Taiwan,null
1971-11-14,South Korea,Vietnam Republic,South Korea,null
1972-05-07,South Korea,Iraq,Iraq,null
1972-05-17,Thailand,South Korea,South Korea,null
1972-05-19,Thailand,Cambodia,Thailand,null
1973-04-21,Senegal,Ghana,Ghana,null
1973-06-14,Guinea,Mali,Guinea,null
1973-06-14,Mauritius,Tanzania,Mauritius,null
1973-07-26,Malaysia,Kuwait,Malaysia,null
1973-07-26,Cambodia,Singapore,Singapore,null


In [0]:
%sql SELECT * FROM bronze.goalscorers LIMIT 10

date,home_team,away_team,team,scorer,minute,own_goal,penalty
1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44,false,false
1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55,false,false
1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70,false,false
1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75,false,false
1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,2,false,false
1916-07-06,Argentina,Chile,Chile,Telésforo Báez,44,false,false
1916-07-06,Argentina,Chile,Argentina,Juan Domingo Brown,60,false,true
1916-07-06,Argentina,Chile,Argentina,Juan Domingo Brown,62,false,true
1916-07-06,Argentina,Chile,Argentina,Alberto Marcovecchio,67,false,false
1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,75,false,false


In [0]:
%sql SELECT * FROM bronze.former_names LIMIT 10

current,former,start_date,end_date
Benin,Dahomey,1959-11-08,1975-11-30
Burkina Faso,Upper Volta,1960-04-14,1984-08-04
Curaçao,Netherlands Antilles,1957-03-03,2010-10-10
Czechoslovakia,Bohemia,1903-04-05,1919-01-01
Czechoslovakia,Bohemia and Moravia,1939-01-01,1940-01-01
DR Congo,Belgian Congo,1948-05-25,1956-01-02
DR Congo,Congo-Léopoldville,1963-04-12,1964-07-19
DR Congo,Congo-Kinshasa,1965-01-09,1970-11-24
DR Congo,Zaïre,1971-01-10,1997-04-27
Djibouti,French Somaliland,1947-12-05,1977-06-27


In [0]:
%sql SELECT * FROM bronze.fifa_ranking LIMIT 10

rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
140.0,Brunei Darussalam,BRU,2.0,0.0,140,AFC,1992-12-31
33.0,Portugal,POR,38.0,0.0,33,UEFA,1992-12-31
32.0,Zambia,ZAM,38.0,0.0,32,CAF,1992-12-31
31.0,Greece,GRE,38.0,0.0,31,UEFA,1992-12-31
30.0,Algeria,ALG,39.0,0.0,30,CAF,1992-12-31
29.0,Yugoslavia,YUG,39.0,0.0,29,UEFA,1992-12-31
28.0,Wales,WAL,40.0,0.0,28,UEFA,1992-12-31
27.0,Côte d'Ivoire,CIV,41.0,0.0,27,CAF,1992-12-31
34.0,Austria,AUT,38.0,0.0,34,UEFA,1992-12-31
26.0,Bulgaria,BUL,41.0,0.0,26,UEFA,1992-12-31


De modo a facilitar o entendimento dos dados no futuro e melhoria da gestão e qualidade dos dados, são incluídas as descrições de cada coluna nas tabelas da camada bronze.

**Tabela Results**

In [0]:
# Adicionar descrições às colunas da tabela bronze_results
table_name = "bronze.results"

spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `date` `date` DATE COMMENT 'Data da partida'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `home_team` `home_team` STRING COMMENT 'Nome do time da casa'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `away_team` `away_team` STRING COMMENT 'Nome do time visitante'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `home_score` `home_score` INT COMMENT 'Pontuação do time da casa (incluindo prorrogação, mas não pênaltis)'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `away_score` `away_score` INT COMMENT 'Pontuação do time visitante (incluindo prorrogação, mas não pênaltis)'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `tournament` `tournament` STRING COMMENT 'Nome do torneio ou competição'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `city` `city` STRING COMMENT 'Nome da cidade onde a partida foi realizada'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `country` `country` STRING COMMENT 'Nome do país onde a partida foi realizada'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `neutral` `neutral` BOOLEAN COMMENT 'Indica se a partida foi realizada em um local neutro (TRUE/FALSE)'")

# Verificar se os comentários foram adicionados
desc_table = spark.sql(f"DESCRIBE EXTENDED {table_name}")
desc_table.show(truncate=False)

+----------------------------+-------------------------------------------+---------------------------------------------------------------------+
|col_name                    |data_type                                  |comment                                                              |
+----------------------------+-------------------------------------------+---------------------------------------------------------------------+
|date                        |date                                       |Data da partida                                                      |
|home_team                   |string                                     |Nome do time da casa                                                 |
|away_team                   |string                                     |Nome do time visitante                                               |
|home_score                  |int                                        |Pontuação do time da casa (incluindo prorrogação, mas nã

**Tabela Shootouts**

In [0]:
table_name = "bronze.shootouts"

spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `date` `date` DATE COMMENT 'Data da partida'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `home_team` `home_team` STRING COMMENT 'Nome do time da casa'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `away_team` `away_team` STRING COMMENT 'Nome do time visitante'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `winner` `winner` STRING COMMENT 'Vencedor da disputa de pênaltis'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `first_shooter` `first_shooter` STRING COMMENT 'Time que chutou primeiro na disputa de pênaltis'")

# Verificar se os comentários foram adicionados
desc_table = spark.sql(f"DESCRIBE EXTENDED {table_name}")
desc_table.show(truncate=False)

+----------------------------+---------------------------------------------------+-----------------------------------------------+
|col_name                    |data_type                                          |comment                                        |
+----------------------------+---------------------------------------------------+-----------------------------------------------+
|date                        |date                                               |Data da partida                                |
|home_team                   |string                                             |Nome do time da casa                           |
|away_team                   |string                                             |Nome do time visitante                         |
|winner                      |string                                             |Vencedor da disputa de pênaltis                |
|first_shooter               |string                                             |T

**Tabela Goalscorers**

In [0]:
table_name = "bronze.goalscorers"

spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `date` `date` DATE COMMENT 'Data da partida'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `home_team` `home_team` STRING COMMENT 'Nome do time da casa'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `away_team` `away_team` STRING COMMENT 'Nome do time visitante'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `team` `team` STRING COMMENT 'Nome do time que marcou o gol'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `scorer` `scorer` STRING COMMENT 'Nome do jogador que marcou o gol'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `own_goal` `own_goal` BOOLEAN COMMENT 'Indica se o gol foi contra (TRUE/FALSE)'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `penalty` `penalty` BOOLEAN COMMENT 'Indica se o gol foi de pênalti (TRUE/FALSE)'")

# Verificar se os comentários foram adicionados
desc_table = spark.sql(f"DESCRIBE EXTENDED {table_name}")
desc_table.show(truncate=False)

+----------------------------+-----------------------------------------------+-------------------------------------------+
|col_name                    |data_type                                      |comment                                    |
+----------------------------+-----------------------------------------------+-------------------------------------------+
|date                        |date                                           |Data da partida                            |
|home_team                   |string                                         |Nome do time da casa                       |
|away_team                   |string                                         |Nome do time visitante                     |
|team                        |string                                         |Nome do time que marcou o gol              |
|scorer                      |string                                         |Nome do jogador que marcou o gol           |
|minute         

**Tabela Former Names**

In [0]:
table_name = "bronze.former_names"

spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `current` `current` STRING COMMENT 'Nome atual do time'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `former` `former` STRING COMMENT 'Nome antigo do time'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `start_date` `start_date` DATE COMMENT 'Data de início do uso do nome antigo'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `end_date` `end_date` DATE COMMENT 'Data de término do uso do nome antigo'")

# Verificar se os comentários foram adicionados
desc_table = spark.sql(f"DESCRIBE EXTENDED {table_name}")
desc_table.show(truncate=False)

+----------------------------+---------------------------------------------------+-------------------------------------+
|col_name                    |data_type                                          |comment                              |
+----------------------------+---------------------------------------------------+-------------------------------------+
|current                     |string                                             |Nome atual do time                   |
|former                      |string                                             |Nome antigo do time                  |
|start_date                  |date                                               |Data de início do uso do nome antigo |
|end_date                    |date                                               |Data de término do uso do nome antigo|
|                            |                                                   |                                     |
|# Detailed Table Information|  

**Tabela FIFA Ranking**

In [0]:
table_name = "bronze.fifa_ranking"

spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `country_full` `country_full` STRING COMMENT 'Nome completo do país'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `country_abrv` `country_abrv` STRING COMMENT 'Abreviação do país'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `rank` `rank` DOUBLE COMMENT 'Posição no ranking FIFA'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `total_points` `total_points` DOUBLE COMMENT 'Pontuação total no ranking FIFA'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `previous_points` `previous_points` DOUBLE COMMENT 'Pontuação anterior no ranking FIFA'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `rank_change` `rank_change` INT COMMENT 'Mudança na posição do ranking desde a última publicação'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `confederation` `confederation` STRING COMMENT 'Confederação FIFA do país'")
spark.sql(f"ALTER TABLE {table_name} CHANGE COLUMN `rank_date` `rank_date` DATE COMMENT 'Data do cálculo do ranking'")

# Verificar se os comentários foram adicionados
desc_table = spark.sql(f"DESCRIBE EXTENDED {table_name}")
desc_table.show(truncate=False)

+----------------------------+------------------------------------------------+-------------------------------------------------------+
|col_name                    |data_type                                       |comment                                                |
+----------------------------+------------------------------------------------+-------------------------------------------------------+
|rank                        |double                                          |Posição no ranking FIFA                                |
|country_full                |string                                          |Nome completo do país                                  |
|country_abrv                |string                                          |Abreviação do país                                     |
|total_points                |double                                          |Pontuação total no ranking FIFA                        |
|previous_points             |double            

Após esse passo, está conluída a construção da camada bronze, que contém os dados brutos e os metadados, de acordo com boas práticas de gestão e governança dos dados. A partir disso, é construída a camada silver, com limpeza, estruturação e preparação dos dados para uso futuro. A camada silver é construída no notebook MVP - Engenharia de Dados - Camada silver